## Load and Store BetterDazls model

We used this code to load and store the dazls model into a .sav file.

We used it in openstef for the components forecast in the file: create_component_forecast, function: create_components_forecast_pipeline

In [ ]:
import pandas as pd
import os
import joblib
import glob
from openstef.model.regressors.better_dazls import BetterDazls


path = "C:\\Users\\AL25802\\new energy splitting"
folder = ['\\prep_data\\']
combined_data = []
station_name = []


# Read prepared data
for file_name in glob.glob(path + folder[0] + '*.csv'):
    x = pd.read_csv(file_name, low_memory=False, parse_dates=["datetime"])
    x["datetime"] = pd.to_datetime(x["datetime"])
    x = x.set_index('datetime')
    x.columns=[x.lower() for x in x.columns]
    combined_data.append(x)
    sn = os.path.basename(file_name)
    station_name.append(sn[:len(sn) - 4])


# Split data in train and test (the first substation is being used for the testing)
training_data = pd.concat(combined_data[1:])
test_data = combined_data[0]
target_columns =['total_solar_part', 'total_wind_part']
feature_columns = [x for x in test_data.columns if x not in target_columns]
print('Testing station:',station_name[0])

# Initialize DAZLS model
model = BetterDazls()
# Fit model
model.fit(training_data.loc[:,feature_columns], training_data.loc[:,target_columns])

# Load and store the model
filename = 'better_dazls_stored.sav'
joblib.dump(model, filename)